In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
from evaluation.main import load_xai_results
from utils import load_json_file
import pandas as pd

config_path = "/home/hjall/work/qai/xai/xai-nlp-benchmark/artifacts/xai-nlp-benchmark-2024-04-23-21-20-02/configs/sentiment_project_config.json"
config = load_json_file(config_path)

/home/hjall/work/qai/xai/xai-nlp-benchmark/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
xai_results = load_xai_results(config)

Loading XAI results:   0%|          | 0/100 [00:00<?, ?it/s]

Loading XAI results: 100%|██████████| 100/100 [01:55<00:00,  1.15s/it]


In [4]:
df = pd.DataFrame(xai_results)
df

,model_name,model_version,model_repetition_number,dataset_type,target,attribution_method,sentence,raw_attribution,attribution,ground_truth,sentence_idx,pred_probabilities
0,one_layer_attention_classification,best,0,gender_all,0.0,Saliency,"['She', 'appears', 'attracted', 'to', 'Rose', ...","[0.024406487, 0.019307744, 0.011081872, 0.0594...","[0.01930774375796318, 0.011081871576607227, 0....","[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[0.6041043400764465, 0.39589565992355347]"
1,one_layer_attention_classification,best,0,gender_all,0.0,InputXGradient,"['She', 'appears', 'attracted', 'to', 'Rose', ...","[0.3106586, 0.00086647633, 0.0019412833, 0.008...","[0.0008664763299748302, 0.001941283349879086, ...","[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[0.6041043400764465, 0.39589565992355347]"
2,one_layer_attention_classification,best,0,gender_all,0.0,Guided Backprop,"['She', 'appears', 'attracted', 'to', 'Rose', ...","[0.042906664, 0.025834529, 0.01156062, 0.00095...","[0.025834528729319572, 0.011560619808733463, 0...","[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[0.6041043400764465, 0.39589565992355347]"
3,one_layer_attention_classification,best,0,gender_all,0.0,Gradient SHAP,"['She', 'appears', 'attracted', 'to', 'Rose', ...","[0.0, 0.025540058, 0.01076425, 0.016589757, 0....","[0.025540057569742203, 0.010764249600470066, 0...","[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[0.6041043400764465, 0.39589565992355347]"
4,one_layer_attention_classification,best,0,gender_all,0.0,LIME,"['She', 'appears', 'attracted', 'to', 'Rose', ...","[0.0, 0.03878744057058616, 0.01325990854352933...","[0.03878744057058616, 0.013259908543529333, 0....","[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[0.6041043400764465, 0.39589565992355347]"
...,...,...,...,...,...,...,...,...,...,...,...,...
643995,bert_all,last,4,gender_subj,1.0,DeepLift,"['David', 'is', 'also', 'representative', 'of'...","[0.016563147, 0.016563147, 0.004140787, 0.0828...","[0.01656314730644226, 0.004140786826610565, 0....","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",321,"[0.01984201930463314, 0.9801580309867859]"
643996,bert_all,last,4,gender_subj,1.0,Kernel SHAP,"['David', 'is', 'also', 'representative', 'of'...","[0.003929022, 0.04394498, 0.034029465, 0.03265...","[0.04394498094916344, 0.03402946516871452, 0.0...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",321,"[0.01984201930463314, 0.9801580309867859]"
643997,bert_all,last,4,gender_subj,1.0,Uniform random,"['David', 'is', 'also', 'representative', 'of'...","[0.049930427, 0.01412816, 0.026240772, 0.02965...","[0.01412815973162651, 0.026240771636366844, 0....","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",321,"[0.01984201930463314, 0.9801580309867859]"
643998,bert_all,last,4,gender_subj,1.0,Integrated Gradients,"['David', 'is', 'also', 'representative', 'of'...","[0.0, 0.017793188468488142, 0.0022616494862824...","[0.017793188468488142, 0.0022616494862824315, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",321,"[0.01984201930463314, 0.9801580309867859]"


In [5]:
df.columns

Index(['model_name', 'model_version', 'model_repetition_number',
       'dataset_type', 'target', 'attribution_method', 'sentence',
       'raw_attribution', 'attribution', 'ground_truth', 'sentence_idx',
       'pred_probabilities'],
      dtype='object')

In [6]:
from tqdm import tqdm
import ast

pred_diffs = []
attribution_diffs = []
attribution_diffs_gt = []
attribution_diffs_not_gt = []

group_columns = [
    'model_name',
    'model_version',
    'model_repetition_number',
    'dataset_type',
    'attribution_method',
    'sentence_idx',
]
for keys, group in tqdm(df.groupby(group_columns)):
    info = {key: value for key, value in zip(group_columns, keys)}

    female = group[group["target"] == 0].iloc[0]
    male = group[group['target'] == 1].iloc[0]

    pred_diff = female["pred_probabilities"][0] - male["pred_probabilities"][0]
    pred_diffs.append({**info, "pred_diff": pred_diff})

    for female_word, male_word, female_attribution, male_attribution, gt in zip(
        ast.literal_eval(female["sentence"]),
        ast.literal_eval(male["sentence"]),
        female["attribution"],
        male["attribution"],
        female["ground_truth"],
    ):
        attribution_diff = female_attribution - male_attribution

        diff_obj = {
            **info,
            "female_word": female_word.lower(),
            "male_word": male_word.lower(),
            "attribution_diff": attribution_diff,
        }

        attribution_diffs.append(diff_obj)

        if gt:
            attribution_diffs_gt.append(diff_obj)
        else:
            attribution_diffs_not_gt.append(diff_obj)

pred_diffs_df = pd.DataFrame(pred_diffs)
attribution_diffs_df = pd.DataFrame(attribution_diffs)
attribution_diffs_gt_df = pd.DataFrame(attribution_diffs_gt)
attribution_diffs_not_gt_df = pd.DataFrame(attribution_diffs_not_gt)

100%|██████████| 322000/322000 [05:58<00:00, 899.05it/s]


In [7]:
# import pickle
# with open("diffs.pkl", "wb") as f:
#     pickle.dump({
#         "pred_diffs_df": pred_diffs_df,
#         "attribution_diffs_df": attribution_diffs_df,
#         "attribution_diffs_gt_df": attribution_diffs_gt_df,
#         "attribution_diffs_not_gt_df": attribution_diffs_not_gt_df,
#     }, f)

In [8]:
import pickle
with open("diffs.pkl", "rb") as f:
    data = pickle.load(f)

    pred_diffs_df =  data["pred_diffs_df"]
    attribution_diffs_df = data["attribution_diffs_df"]
    attribution_diffs_gt_df = data["attribution_diffs_gt_df"]
    attribution_diffs_not_gt_df = data["attribution_diffs_not_gt_df"]

In [9]:
pred_diffs_df

,model_name,model_version,model_repetition_number,dataset_type,attribution_method,sentence_idx,pred_diff
0,bert_all,best,0,gender_all,Correlation,0,0.088756
1,bert_all,best,0,gender_all,Correlation,1,0.013950
2,bert_all,best,0,gender_all,Correlation,2,-0.028580
3,bert_all,best,0,gender_all,Correlation,3,0.018108
4,bert_all,best,0,gender_all,Correlation,4,-0.035677
...,...,...,...,...,...,...,...
321995,one_layer_attention_classification,last,4,gender_subj,Uniform random,317,-0.033364
321996,one_layer_attention_classification,last,4,gender_subj,Uniform random,318,-0.185438
321997,one_layer_attention_classification,last,4,gender_subj,Uniform random,319,0.163517
321998,one_layer_attention_classification,last,4,gender_subj,Uniform random,320,0.011413


In [9]:
pred_diffs_df.columns

Index(['model_name', 'model_version', 'model_repetition_number',
       'dataset_type', 'attribution_method', 'sentence_idx', 'pred_diff'],
      dtype='object')

In [21]:
from scipy import stats

test = "ttest"
test = "wilcoxon"

# Load two arrays of samples
group_by = [
    'model_name',
    'model_version',
    'dataset_type',
    'attribution_method',
	'model_repetition_number'
]

results = []
for keys, group in pred_diffs_df.groupby(group_by):
	info = {key: value for key, value in zip(group_by, keys)}
	diff = group["pred_diff"].values

	alpha = 0.05
	if test == "ttest":
		mu = 0
		t_stat, p_value = stats.ttest_1samp(diff, mu)

		results.append({
			**info,
			"t_stat": t_stat,
			"p_value": p_value.round(4),
			"reject": p_value < alpha,
		})
	elif test == "wilcoxon":
		w_stat, p_value = stats.wilcoxon(diff)

		results.append({
			**info,
			"w_stat": w_stat,
			"p_value": p_value.round(4),
			"reject": p_value < alpha,
		})

pd.DataFrame(results)

,model_name,model_version,dataset_type,attribution_method,model_repetition_number,w_stat,p_value,reject
0,bert_all,best,gender_all,Correlation,0,14533.0,0.0000,True
1,bert_all,best,gender_all,Correlation,1,18097.0,0.0000,True
2,bert_all,best,gender_all,Correlation,2,23045.0,0.0770,False
3,bert_all,best,gender_all,Correlation,3,19611.0,0.0001,True
4,bert_all,best,gender_all,Correlation,4,21048.0,0.0030,True
...,...,...,...,...,...,...,...,...
995,one_layer_attention_classification,last,gender_subj,Uniform random,0,23666.0,0.1913,False
996,one_layer_attention_classification,last,gender_subj,Uniform random,1,12235.0,0.0000,True
997,one_layer_attention_classification,last,gender_subj,Uniform random,2,22648.0,0.0551,False
998,one_layer_attention_classification,last,gender_subj,Uniform random,3,18511.0,0.0000,True


In [18]:
attribution_diffs_df

,model_name,model_version,model_repetition_number,dataset_type,attribution_method,sentence_idx,female_word,male_word,attribution_diff
0,bert_all,best,0,gender_all,Correlation,0,she,he,0.011732
1,bert_all,best,0,gender_all,Correlation,0,appears,appears,0.000004
2,bert_all,best,0,gender_all,Correlation,0,attracted,attracted,0.000002
3,bert_all,best,0,gender_all,Correlation,0,to,to,0.000038
4,bert_all,best,0,gender_all,Correlation,0,rose,flynn,-0.026332
...,...,...,...,...,...,...,...,...,...
6056995,one_layer_attention_classification,last,4,gender_subj,Uniform random,321,allowed,allowed,0.004482
6056996,one_layer_attention_classification,last,4,gender_subj,Uniform random,321,to,to,-0.010597
6056997,one_layer_attention_classification,last,4,gender_subj,Uniform random,321,engage,engage,0.018385
6056998,one_layer_attention_classification,last,4,gender_subj,Uniform random,321,in,in,0.036142


In [23]:
group_by = [
    'model_name',
    'model_version',
    'dataset_type',
    'attribution_method',
	'model_repetition_number'
]

cur_df = attribution_diffs_df
# cur_df = attribution_diffs_gt_df
cur_df = attribution_diffs_not_gt_df

results = []
for keys, group in cur_df.groupby(group_by):
    info = {key: value for key, value in zip(group_by, keys)}
    diff = group["attribution_diff"].values

    alpha = 0.05

    if test == "ttest":
        mu = 0
        t_stat, p_value = stats.ttest_1samp(diff, mu)
        results.append(
            {
                **info,
                "t_stat": t_stat,
                "p_value": p_value,
                "reject": p_value < alpha,
            }
        )
    elif test == "wilcoxon":
        w_stat, p_value = stats.wilcoxon(diff)
        results.append(
            {
                **info,
                "w_stat": w_stat,
                "p_value": p_value,
                "reject": p_value < alpha,
            }
        )

results_df = pd.DataFrame(results)

results_df[results_df["reject"]]

,model_name,model_version,dataset_type,attribution_method,model_repetition_number,w_stat,p_value,reject
0,bert_all,best,gender_all,Correlation,0,2566614.0,1.087317e-110,True
1,bert_all,best,gender_all,Correlation,1,2566614.0,1.087317e-110,True
2,bert_all,best,gender_all,Correlation,2,2566614.0,1.087317e-110,True
3,bert_all,best,gender_all,Correlation,3,2566614.0,1.087317e-110,True
4,bert_all,best,gender_all,Correlation,4,2566614.0,1.087317e-110,True
...,...,...,...,...,...,...,...,...
989,one_layer_attention_classification,last,gender_subj,LIME,4,5600707.0,1.089111e-02,True
990,one_layer_attention_classification,last,gender_subj,Saliency,0,6660640.0,2.422226e-16,True
991,one_layer_attention_classification,last,gender_subj,Saliency,1,5555949.5,1.080967e-68,True
992,one_layer_attention_classification,last,gender_subj,Saliency,2,5600739.0,7.688724e-66,True


In [24]:
results_df[~results_df["reject"]]

,model_name,model_version,dataset_type,attribution_method,model_repetition_number,w_stat,p_value,reject
5,bert_all,best,gender_all,DeepLift,0,6287181.5,0.632136,False
6,bert_all,best,gender_all,DeepLift,1,6265182.0,0.535650,False
7,bert_all,best,gender_all,DeepLift,2,6302791.0,0.818076,False
8,bert_all,best,gender_all,DeepLift,3,6310338.5,0.780636,False
9,bert_all,best,gender_all,DeepLift,4,6257794.5,0.520442,False
...,...,...,...,...,...,...,...,...
995,one_layer_attention_classification,last,gender_subj,Uniform random,0,7647571.0,0.681957,False
996,one_layer_attention_classification,last,gender_subj,Uniform random,1,7596301.0,0.401178,False
997,one_layer_attention_classification,last,gender_subj,Uniform random,2,7657561.5,0.744381,False
998,one_layer_attention_classification,last,gender_subj,Uniform random,3,7648864.0,0.689926,False
